In [1]:
# imports
import requests
import json
import pandas as pd
import os
from pprint import pprint
import sqlite3

# Foursquare

Send a request to Foursquare with location Downtown Vancouver for food near me.

radius: 1000m is walkable in ~10 minutes.

Get the price, loaction, name, rating, distance from me, and number of ratings

In [2]:
FOURSQUARE_KEY = os.environ["FOURSQUARE_API"]

# set params
params = {
        #downtown vancouver lat long
        "query": 'food',
        "ll": '49.28,-123.11',
        "radius": 1000,
        "sort": "DISTANCE",
        "fields": "name,rating,location,distance,price,stats",
        "limit": 50
}
url = "https://api.foursquare.com/v3/places/search"

# Create dictionary for headers
headers = {"Accept": "application/json"}
# Add key with our API KEY
headers['Authorization'] = FOURSQUARE_KEY

In [3]:
# send request to Foursquare
res = requests.get(url=url, params=params, headers=headers)
fdata = res.json()


In [4]:
# save results to file
with open('foursquare.json','w') as f:
    json.dump(fdata, f)


In [5]:
# print the results
pprint(fdata)


{'context': {'geo_bounds': {'circle': {'center': {'latitude': 49.28,
                                                  'longitude': -123.11},
                                       'radius': 1000}}},
 'results': [{'distance': 10,
              'location': {'address': '568 Beatty St',
                           'country': 'CA',
                           'cross_street': 'btwn W Pender & Dunsmuir',
                           'formatted_address': '568 Beatty St (btwn W Pender '
                                                '& Dunsmuir), Vancouver BC V6B '
                                                '2L3',
                           'locality': 'Vancouver',
                           'neighborhood': ['Gastown'],
                           'postcode': 'V6B 2L3',
                           'region': 'BC'},
              'name': 'Chambar',
              'price': 3,
              'rating': 8.9,
              'stats': {'total_photos': 547,
                        'total_ratings': 517,
   

Parse through the response to get the POI details you want (rating, name, location, etc)

In [6]:
four_dict = {
              'name': [],
              'address': [],
              'distance': [],
              'price': [],
              'rating': [],
              'number_ratings': []
}

def price_change(fl):
    if fl == 1.0:
        return '$'
    elif fl == 2.0:
        return '$$'
    elif fl == 3.0:
        return '$$$'

for result in fdata['results']:
    four_dict['name'].append(result['name'])
    four_dict['address'].append(result['location']['address'])
    four_dict['rating'].append(result['rating'])
    four_dict['distance'].append(result['distance'])
    try:
        # convert price to $ like yelp
        four_dict['price'].append(price_change(result['price']))
    except:
        four_dict['price'].append(None)
    four_dict['number_ratings'].append(result['stats']['total_ratings'])

Put your parsed results into a DataFrame

In [7]:
four_places = pd.DataFrame(four_dict, columns= ['name', 'address', 'distance', 'rating', 'price','number_ratings'])
four_places.sort_values(by='number_ratings', ascending=False)

,name,address,distance,rating,price,number_ratings
15,Meat & Bread,370 Cambie St,296,9.2,$$,606
0,Chambar,568 Beatty St,10,8.9,$$$,517
46,Pourhouse Restaurant,162 Water St,484,8.8,$$,379
49,Tacofino,15 Cordova St W,497,8.7,$,304
44,Nelson the Seagull,315 Carrall St,481,8.5,$$,279
16,Nuba,207B W Hastings St,313,8.0,$$,263
22,La Taqueria,322 Hastings St W,337,8.7,$$,212
13,Catch 122,122 Hastings St W,282,7.9,$,197
20,Finch's Tea & Coffee House,353 Pender St W,333,8.6,$$,180
2,Jam Cafe,556 Beatty St,45,8.7,$,147


In [8]:
# get the average number of ratings
average_num_ratings_f = four_places['number_ratings'].sum() / four_places.shape[0]
print(average_num_ratings_f)

107.02


# Yelp

Send a request with the same paramaters (location, radius, etc) to see which API is better at getting information wanted.

In [9]:
YELP_KEY = os.environ["YELP_API"]
# print(YELP_KEY)

# set params
params = {
        #downtown vancouver lat long
        "term": 'food',
        "latitude": '49.28',
        "longitude": '-123.11',
        "radius": 1000,
        "sort_by": "distance",
        "limit": 50
}
url = "https://api.yelp.com/v3/businesses/search"

# Create dictionary for headers
headers = {"Accept": "application/json"}
# Add key with our API KEY
headers['Authorization'] = f"Bearer {YELP_KEY}"

In [10]:
# send request to YELP
res = requests.get(url=url, params=params, headers=headers)
ydata = res.json()
res.status_code

200

In [11]:
# save results to file
with open('yelp.json','w') as f:
    json.dump(ydata, f)

In [12]:
pprint(ydata)

{'businesses': [{'alias': 'chambar-vancouver',
                 'categories': [{'alias': 'belgian', 'title': 'Belgian'}],
                 'coordinates': {'latitude': 49.2801460007961,
                                 'longitude': -123.109925328073},
                 'display_phone': '+1 604-879-7119',
                 'distance': 17.114322140412565,
                 'id': '_4R46MNkwx9MeOyt0YfNxA',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/fSzNu6lZuyOz65t229txpw/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '568 Beatty Street',
                              'address2': '',
                              'address3': '',
                              'city': 'Vancouver',
                              'country': 'CA',
                              'display_address': ['568 Beatty Street',
                                                  'Vancouver, BC V6B 2L3',
                                                  'Canada'],

Parse through your result and get POI details

In [13]:
yelp_dict = {
              'name': [],
              'address': [],
              'distance': [],
              'rating': [],
              'price': [],
              'number_ratings': []
              
}

for result in ydata['businesses']:
    yelp_dict['name'].append(result['name'])
    yelp_dict['address'].append(result['location']['address1'])
    yelp_dict['distance'].append(int(result['distance']))
    yelp_dict['rating'].append(float(result['rating']))
    try:
        yelp_dict['price'].append(result['price'])
    except:
        yelp_dict['price'].append(None)
    yelp_dict['number_ratings'].append(result['review_count'])

    

Put your parsed results into a DataFrame

In [14]:
yelp_places = pd.DataFrame(yelp_dict)
yelp_places.sort_values(by='number_ratings', ascending= False)

,name,address,distance,rating,price,number_ratings
0,Chambar,568 Beatty Street,17,4.0,$$$,1479
5,Jam Cafe on Beatty,556 Beatty Street,37,4.5,$$,1246
17,Taishoken Ramen,515 Abbott Street,172,3.5,$$,251
18,TAKO,601 Expo Boulevard,178,4.0,$,209
37,Tsuki Sushi Bar,509 Abbott Street,201,4.0,$$,190
3,Devil's Elbow Ale & Smoke House,562 Beatty Street,31,3.5,$$,188
11,Pizzeria Ludica,189 Keefer Place,109,4.0,$$,177
20,Sushi Den,609 Abbott Street,179,3.0,$$,161
12,T & T Supermarket,179 Keefer Place,115,3.5,$$,138
38,Catfe,International Village Mall,206,3.5,$$,96


In [15]:
# get the average number of ratings
average_num_ratings_y = yelp_places['number_ratings'].sum() / yelp_places.shape[0]
print(average_num_ratings_y)

97.82


### Which API is better?

Both APIS have a limit of 50 items per search.

Decided to calculate the average number of reviews per entry to see which service had more use.



In [16]:
print(f" Foursquare API averages {average_num_ratings_f} ratings over {four_places.shape[0]} entries")

print(f" Yelp API average {average_num_ratings_y} ratings over {yelp_places.shape[0]} entries")

 Foursquare API averages 107.02 ratings over 50 entries
 Yelp API average 97.82 ratings over 50 entries


# Database

### Connect and create a SQL database


In [17]:
# create database
places = sqlite3.connect('places_database.db')
p = places.cursor()

In [18]:
p.execute('CREATE TABLE IF NOT EXISTS yelp_places (name text, address text, rating double, price text)')
p.execute('CREATE TABLE IF NOT EXISTS four_places (name text, address text, rating double, price text)')
places.commit()

In [19]:
yelp_places.to_sql('yelp_places', places, if_exists = 'replace', index = False)
four_places.to_sql('four_places', places, if_exists='replace', index=False)

p.execute('''
SELECT * FROM yelp_places
''')

p.execute('''
SELECT * FROM four_places
''')


